# Processing big Sentinel-1 SAR data in python using snappy

- You learn how to use snappy to apply standard pre-processing chain and produce Sentinel-1 analysis ready data.

    1) Download and install latest SNAP release on Windows,
    2) Configure snappy during and after the SNAP installation process
    3) Setup a virtual environment,
    4) Configure optimal settings for snappy,
    5) Visualize satellite data using snappy

In [3]:
import sys
#sys.path.append('C:\Users\vaak\.snap\auxdata\snaphu-v1.4.2_win64\bin')
sys.path.append('C:\\Users\\vaak\\.snap\\snap-python\snappy')
sys.path.append('C:\\Users\\vaak\\Anaconda3\\envs\\snap\\Lib')

#  Load Python modules

In [4]:
import os
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
from snappy import jpy
import subprocess
from time import *

In [5]:
# Hashmap is used to give us access to all JAVA operators
HashMap = jpy.get_type('java.util.HashMap')
parameters = HashMap()

# Define user functions

In [32]:

def read(filename):
    return ProductIO.readProduct(filename)

def topsar_split(product,IW):
    print('\tApply TOPSAR Split...')
    parameters = HashMap()
    parameters.put('subswath', IW)
    #parameters.put('selectedPolarisations', 'VV')
    output=GPF.createProduct("TOPSAR-Split", parameters, product)
    return output

def apply_orbit_file(product):
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)
    return GPF.createProduct("Apply-Orbit-File", parameters, product)

def back_geocoding(product):
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("DEM Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put("Resampling Type", "BISINC_5_POINT_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", False)
    return GPF.createProduct("Back-Geocoding", parameters, product)

def interferogram(product):
    parameters.put("Subtract flat-earth phase", True)
    parameters.put("Degree of \"Flat Earth\" polynomial", 5)
    parameters.put("Number of \"Flat Earth\" estimation points", 501)
    parameters.put("Orbit interpolation degree", 3)
    parameters.put("Include coherence estimation", True)
    parameters.put("Square Pixel", False)
    parameters.put("Independent Window Sizes", False)
    parameters.put("Coherence Azimuth Window Size", 10)
    parameters.put("Coherence Range Window Size", 10)
    return GPF.createProduct("Interferogram", parameters, product)

def topsar_deburst(source):  
    parameters = HashMap()
    parameters.put("Polarisations", "VV,VH")
    output=GPF.createProduct("TOPSAR-Deburst", parameters, source)
    return output

def topophase_removal(product):
    parameters.put("Orbit Interpolation Degree", 3)
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("Tile Extension[%]", 100)
    parameters.put("Output topographic phase band", True)
    parameters.put("Output elevation band", False)
    return GPF.createProduct("TopoPhaseRemoval", parameters, product)

def goldstein_phasefiltering(product):
    parameters.put("Adaptive Filter Exponent in(0,1]:", 1.0)
    parameters.put("FFT Size", 64)
    parameters.put("Window Size", 3)
    parameters.put("Use coherence mask", False)
    parameters.put("Coherence Threshold in[0,1]:", 0.2)
    return GPF.createProduct("GoldsteinPhaseFiltering", parameters, product)
def write(product, filename):
    ProductIO.writeProduct(product, filename, "GeoTiff")

In [34]:
def insar_pipeline(filename_1, filename_2,IW,outputpath):
    print('Reading SAR data')
    product_1 = read(filename_1)
    product_2 = read(filename_2)
    print('TOPSAR-Split')
    product_TOPSAR_1 = topsar_split(product_1,IW)
    product_TOPSAR_2 = topsar_split(product_2,IW)
    print('Applying precise orbit files')
    product_orbitFile_1 = apply_orbit_file(product_TOPSAR_1)
    product_orbitFile_2 = apply_orbit_file(product_TOPSAR_2)
    print('back geocoding')
    backGeocoding = back_geocoding([product_orbitFile_1, product_orbitFile_2])
    print('inerferogram generation')
    interferogram_formation = interferogram(backGeocoding)
    print('TOPSAR_deburst')
    TOPSAR_deburst = topsar_deburst(interferogram_formation)
    print('TopoPhase removal')
    TOPO_phase_removal =topophase_removal(TOPSAR_deburst)
    print('Goldstein filtering')
    goldstein_phasefiltering(TOPO_phase_removal)
    print('Writing final product')
    write(TOPO_phase_removal, outputpath)

In [35]:
filename_1 = os.path.join(r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\DSC','S1A_IW_SLC__1SDV_20200119T004047_20200119T004114_030862_038AA2_B3C2.zip')
filename_2 = os.path.join(r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\DSC','S1A_IW_SLC__1SDV_20190124T004042_20190124T004110_025612_02D7DC_531F.zip')
outpath =os.path.join(r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad','InSAR')


In [ ]:
IW='IW2'
insar_pipeline(filename_1, filename_2,IW,outpath)

Reading SAR data
TOPSAR-Split
	Apply TOPSAR Split...
	Apply TOPSAR Split...
Applying precise orbit files
back geocoding


In [17]:
def main():
    ## All Sentinel-1 data sub folders are located within a super folder (make sure the data is already unzipped and each sub folder name ends with '.SAFE'):
    path=r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\DSC'
    outpath = r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\sigma0'
    if not os.path.exists(outpath):
        os.makedirs(outpath)
        
    LONMIN=str(76.3382)
    LATMIN=str(9.489021)
    LONMAX=str(76.43433)
    LATMAX=str(9.687060)
    IW='IW2'
    
    GRes=10
    
    #LONMIN=str(78.2584)
    #LATMIN=str(17.5185)
    #LONMAX=str(78.2680)
    #LATMAX=str(17.5296)
    
   # 'POLYGON((78.2584 17.5296,78.2584 17.5185,78.2632 17.5169,78.2680 17.5185,78.2684 17.5224,78.2636 17.5294,78.2584 17.5296))'
  
    wkt='POLYGON (('+LONMIN+' '+LATMIN+','+LONMAX+' '+LATMIN+','+LONMAX+' '+LATMAX+','+LONMIN+' '+LATMAX+','+LONMIN+' '+LATMIN+'))'

    #wkt = 'POLYGON ((76.3834 9.6730149691062, 76.41729354858398 9.6730149691062, \
    #76.41729354858398 9.628506952098645, 76.3834 9.628506952098645, 76.3834 9.6730149691062))'
        
    ## UTM projection parameters
    proj = '''PROJCS["WGS 84 / Auto UTM",GEOGCS["World Geodetic System 1984",
    DATUM["World Geodetic System 1984",SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
    AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],UNIT["degree", 0.017453292519943295],
    AXIS["Geodetic longitude", EAST],AXIS["Geodetic latitude", NORTH]],PROJECTION["Transverse_Mercator"],
    PARAMETER["central_meridian", 75.0],PARAMETER["latitude_of_origin", 0.0],
    PARAMETER["scale_factor", 0.9996],PARAMETER["false_easting", 500000.0],
    PARAMETER["false_northing", 0.0],UNIT["m", 1.0],AXIS["Easting", EAST],AXIS["Northing", NORTH]]'''
    

    
    

    for folder in os.listdir(path):
        gc.enable()
        gc.collect()
        sentinel_1 = ProductIO.readProduct(path + "\\" + folder)
        print(sentinel_1)

        loopstarttime=str(datetime.datetime.now())
        print('Start time:', loopstarttime)
        start_time = time.time()

        ## Extract mode, product type, and polarizations from filename
        modestamp = folder.split("_")[1]
        productstamp = folder.split("_")[2]
        polstamp = folder.split("_")[4]

        polarization = polstamp[2:4]

        if polarization == 'DV':
            pols = 'VH,VV'
        elif polarization == 'DH':
            pols = 'HH,HV'
        elif polarization == 'SH' or polarization == 'HH':
            pols = 'HH'
        elif polarization == 'SV':
            pols = 'VV'
        else:
            print("Polarization error!")

        print(pols)    
        ## Start preprocessing:
        topsarsplit=topsar_split(sentinel_1,IW)
        applyorbit = do_apply_orbit_file(topsarsplit)
        thermaremoved = do_thermal_noise_removal(applyorbit)
        calibrated = do_calibration(thermaremoved, polarization, pols)
        topsardeburst=topsar_deburst(calibrated)
        filtered = do_speckle_filtering(topsardeburst)
        del topsarsplit
        del applyorbit
        del thermaremoved
        del calibrated
        del topsardeburst
        ## IW images are downsampled from 10m to 40m (the same resolution as EW images).
        tercorrected = do_terrain_correction(filtered, proj, 0)
        subset = do_subset(tercorrected, wkt)
        res_db=convert_dB(subset)
        del filtered
        del tercorrected

        print("Writing...")
        outputname='sigma0_'+folder[17:25]+'_'+IW

        ProductIO.writeProduct(res_db, outpath + '\\' + outputname, 'GeoTIFF')
        #ProductIO.writeProduct(res_db, outpath + '\\' + folder[:-5], 'GeoTIFF')
        del subset
    

        print('Done.')
        sentinel_1.dispose()
        sentinel_1.closeIO()
        print("--- %s seconds ---" % (time.time() - start_time))

if __name__== "__main__":
    main()

NameError: name 'gc' is not defined

In [3]:
def read(filename):
    output=ProductIO.readProduct(filename)
    return output

def topsar_split(product,IW):
    print('\tApply TOPSAR Split...')
    parameters = HashMap()
    parameters.put('subswath', IW)
    #parameters.put('selectedPolarisations', 'VV')
    output=GPF.createProduct("TOPSAR-Split", parameters, product)
    return output

def do_apply_orbit_file(source):
    print('\tApply orbit file...')
    parameters = HashMap()
    parameters.put('Apply-Orbit-File', True)
    output = GPF.createProduct('Apply-Orbit-File', parameters, source)
    return output

def do_thermal_noise_removal(source):
    print('\tThermal noise removal...')
    parameters = HashMap()
    parameters.put('removeThermalNoise', True)
    output = GPF.createProduct('ThermalNoiseRemoval', parameters, source)
    return output

def do_calibration(source, polarization, pols):
    print('\tCalibration...')
    parameters = HashMap()
    parameters.put('outputSigmaBand', True)
    parameters.put('outputImageInComplex', True)

    #if polarization == 'DH':
     #   parameters.put('sourceBands', 'Intensity_HH,Intensity_HV')
    #elif polarization == 'DV':
     #   parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    #elif polarization == 'SH' or polarization == 'HH':
     #   parameters.put('sourceBands', 'Intensity_HH')
    #elif polarization == 'SV':
     #   parameters.put('sourceBands', 'Intensity_VV')
    #else:
     #   print("different polarization!")
    #parameters.put('selectedPolarisations', pols)
    #parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, source)
    return output

def do_speckle_filtering(source):
    print('\tSpeckle filtering...')
    parameters = HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    output = GPF.createProduct('Speckle-Filter', parameters, source)
    return output

def topsar_deburst(source):  
    parameters = HashMap()
    parameters.put("Polarisations", "VV,VH")
    output=GPF.createProduct("TOPSAR-Deburst", parameters, source)
    return output

def do_terrain_correction(source, proj, downsample):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'SRTM 3Sec')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84
    parameters.put('saveProjectedLocalIncidenceAngle', False)
    parameters.put('saveSelectedSourceBand', True)
    parameters.put('nodataValueAtSea', False)
    parameters.put('pixelSpacingInMeter', 10.0)

    while downsample == 1:                      # downsample: 1 -- need downsample to 40m, 0 -- no need to downsample
        parameters.put('pixelSpacingInMeter', 40.0)
        break
    output = GPF.createProduct('Terrain-Correction', parameters, source)
    return output

def do_subset(source, wkt):
    print('\tSubsetting...')
    parameters = HashMap()
    parameters.put('geoRegion', wkt)
    #parameters.put('outputImageScaleInDb', True)
    output = GPF.createProduct('Subset', parameters, source)
    return output

def convert_dB(source):
    print('\converting to db…')
    parameters = HashMap()
    output = GPF.createProduct('LinearToFromdB', parameters, source)
    return output

def polarimetric_mat(source):
    print('\converting to db…')
    parameters = HashMap()
    parameters.put('matrix', 'C2')
    output = GPF.createProduct('Polarimetric-Matrices', parameters, source)
    return output

def multilook(source,rg,az):
        print('\multilooking…')
        parameters      = HashMap() 
        parameters.put("grSquarePixel", True)
        parameters.put("outputIntensity", False)
        parameters.put("nRgLooks", rg)
        parameters.put("nAzLooks", az)
        output     =  GPF.createProduct("Multilook", parameters, source)
        return output


In [4]:
def main():
    ## All Sentinel-1 data sub folders are located within a super folder (make sure the data is already unzipped and each sub folder name ends with '.SAFE'):
    #path = r'S:\Prosjekter\Landsskog\Vahid\India\DSC'
    path=r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\DSC'
    outpath = r'C:\Users\vaak\Desktop\UoS\weedwatch project\S1_Vembanad\geoMLC'
    if not os.path.exists(outpath):
        os.makedirs(outpath)
        
    LONMIN=str(76.3382)
    LATMIN=str(9.489021)
    LONMAX=str(76.43433)
    LATMAX=str(9.687060)
    IW='IW2'
    
    #LONMIN=str(78.2584)
    #LATMIN=str(17.5185)
    #LONMAX=str(78.2680)
    #LATMAX=str(17.5296)
    
   # 'POLYGON((78.2584 17.5296,78.2584 17.5185,78.2632 17.5169,78.2680 17.5185,78.2684 17.5224,78.2636 17.5294,78.2584 17.5296))'
  
    wkt='POLYGON (('+LONMIN+' '+LATMIN+','+LONMAX+' '+LATMIN+','+LONMAX+' '+LATMAX+','+LONMIN+' '+LATMAX+','+LONMIN+' '+LATMIN+'))'

    #wkt = 'POLYGON ((76.3834 9.6730149691062, 76.41729354858398 9.6730149691062, \
    #76.41729354858398 9.628506952098645, 76.3834 9.628506952098645, 76.3834 9.6730149691062))'
        
    ## UTM projection parameters
    proj = '''PROJCS["WGS 84 / Auto UTM",GEOGCS["World Geodetic System 1984",
    DATUM["World Geodetic System 1984",SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
    AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],UNIT["degree", 0.017453292519943295],
    AXIS["Geodetic longitude", EAST],AXIS["Geodetic latitude", NORTH]],PROJECTION["Transverse_Mercator"],
    PARAMETER["central_meridian", 75.0],PARAMETER["latitude_of_origin", 0.0],
    PARAMETER["scale_factor", 0.9996],PARAMETER["false_easting", 500000.0],
    PARAMETER["false_northing", 0.0],UNIT["m", 1.0],AXIS["Easting", EAST],AXIS["Northing", NORTH]]'''
    

    
    

    for folder in os.listdir(path):
        gc.enable()
        gc.collect()
        sentinel_1 = ProductIO.readProduct(path + "\\" + folder)
        print(sentinel_1)

        loopstarttime=str(datetime.datetime.now())
        print('Start time:', loopstarttime)
        start_time = time.time()

        ## Extract mode, product type, and polarizations from filename
        modestamp = folder.split("_")[1]
        productstamp = folder.split("_")[2]
        polstamp = folder.split("_")[4]

        polarization = polstamp[2:4]

        if polarization == 'DV':
            pols = 'VH,VV'
        elif polarization == 'DH':
            pols = 'HH,HV'
        elif polarization == 'SH' or polarization == 'HH':
            pols = 'HH'
        elif polarization == 'SV':
            pols = 'VV'
        else:
            print("Polarization error!")

        print(pols)    
        ## Start preprocessing:
        topsarsplit=topsar_split(sentinel_1,IW)
        applyorbit = do_apply_orbit_file(topsarsplit)
        #thermaremoved = do_thermal_noise_removal(applyorbit)
        calibrated = do_calibration(applyorbit, polarization, pols)
        topsardeburst=topsar_deburst(calibrated)
        polmat=polarimetric_mat(topsardeburst)
        rg=4
        az=1
        multilooked=multilook(polmat,rg,az)
            
        del topsarsplit
        del applyorbit
        #del thermaremoved
        del calibrated
        del topsardeburst
        del polmat
        
        ## IW images are downsampled from 10m to 40m (the same resolution as EW images).
        tercorrected = do_terrain_correction(multilooked, proj, 0)
        subset = do_subset(tercorrected, wkt)
        res_db=convert_dB(subset)
        del multilooked
        del tercorrected

        print("Writing...")
        outputname='GeoMLC_'+folder[17:25]+'_'+IW

        ProductIO.writeProduct(res_db, outpath + '\\' + outputname, 'GeoTIFF')
        #ProductIO.writeProduct(res_db, outpath + '\\' + folder[:-5], 'GeoTIFF')
        del subset
    

        print('Done.')
        sentinel_1.dispose()
        sentinel_1.closeIO()
        print("--- %s seconds ---" % (time.time() - start_time))

if __name__== "__main__":
    main()

org.esa.snap.core.datamodel.Product[name=S1A_IW_SLC__1SDV_20190124T004042_20190124T004110_025612_02D7DC_531F]
Start time: 2021-02-09 20:49:36.226287
VH,VV
	Apply TOPSAR Split...
	Apply orbit file...
	Calibration...
\converting to db…
\multilooking…
	Terrain correction...
	Subsetting...
\converting to db…
Writing...
Done.
--- 79.11167645454407 seconds ---
org.esa.snap.core.datamodel.Product[name=S1A_IW_SLC__1SDV_20200119T004047_20200119T004114_030862_038AA2_B3C2]
Start time: 2021-02-09 20:50:57.091616
VH,VV
	Apply TOPSAR Split...
	Apply orbit file...
	Calibration...
\converting to db…
\multilooking…
	Terrain correction...
	Subsetting...
\converting to db…
Writing...
Done.
--- 95.03329873085022 seconds ---
